## Get the Workspace

In [1]:
from azureml.core.workspace import Workspace

workspace = Workspace.from_config()

## Set the Compute Target

In [2]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

cluster_name = 'aml-labs-01'
compute_target = ComputeTarget(workspace=workspace, name=cluster_name)

## Create an experiment

In [3]:
from azureml.core import Experiment

experiment_name = 'churn-experiment'
experiment = Experiment(workspace=workspace, name=experiment_name)

## Create Environment

In [10]:
import os
import shutil

project_folder = './xgboost-churn'
os.makedirs(project_folder, exist_ok=True)

# Copy the script to the project folder
shutil.copy('src/train.py', project_folder)

'./xgboost-churn/train.py'

In [4]:
from azureml.core import ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies

xgboost_env = Environment(workspace=workspace, name="XGBoost-Env")
conda_dep = CondaDependencies()

conda_dep.add_pip_package("azureml-dataprep[pandas]")
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("xgboost==0.90")
conda_dep.add_pip_package("scikit-learn==0.22.2.post1")
conda_dep.add_pip_package("numpy==1.18.5")

# Adds dependencies to PythonSection of myenv
xgboost_env.python.conda_dependencies=conda_dep

In [5]:
xgboost_env.register(workspace)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210922.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "XGBoost-Env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [ ]:
# Non optimized Run
src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      arguments=['--max_depth', 30, 
                                 '--learning_rate', 0.5, 
                                 '--reg_alpha', 0.5, 
                                 '--reg_lambda', 0.2, 
                                 '--min_child_weight', 15.00],
                      compute_target=compute_target,
                      environment=xgboost_env)

# Optimized Run
# src = ScriptRunConfig(source_directory=project_folder,
#                       script='train.py',
#                       arguments=['--max_depth', 64, 
#                                  '--learning_rate', 0.2260, 
#                                  '--reg_alpha', 0.01, 
#                                  '--reg_lambda', 0.348, 
#                                  '--min_child_weight', 19.22],
#                       compute_target=compute_target,
#                       environment=xgboost_env)

## Submit the run

In [14]:
run = experiment.submit(src)

## Monitor

In [15]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Register the Model

In [18]:
run.download_file(name='outputs/model.pickle', output_file_path='outputs')

In [19]:
model = run.register_model(model_name='churn-model', 
                           model_path='outputs/model.pickle',
                           description='Churn Model',
                           tags={'Model': 'XGBoost', 'type': 'Classification'},
                          )